In [1]:
print("HEllo world")

HEllo world


In [2]:
import os
from langchain.agents import *
from langchain.llms import OpenAI
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import AgentExecutor

In [5]:
pip install pymysql

                                              0.0/44.8 kB ? eta -:--:--
     ---------------------------------------- 44.8/44.8 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


## DATABASE CONNECTIVITY

In [6]:
db_user='root'
db_password='1234'
db_host='localhost'
db_name='sample_dataset'
db=SQLDatabase.from_uri(f'mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}')

In [12]:

import os
from dotenv import load_dotenv
# Load environment variables from .env file
load_dotenv()

# Access the OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")


In [13]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(model_name='gpt-3.5-turbo', openai_api_key=openai_api_key)

In [14]:
toolkit=SQLDatabaseToolkit(db=db,llm=llm)

In [21]:
agent_executer=create_sql_agent(
    llm=llm,
    toolkit=toolkit
)

In [40]:
agent_executer.run("How many tables do we have?")

'There are 8 tables in the database.'

In [39]:
agent_executer.run("Describe each table")

'The tables in the database are customers, employees, offices, orderdetails, orders, payments, productlines, and products.'

In [23]:
res=agent_executer.run("what are the columns in each table?")

In [24]:
print(res)

The columns in each table are as follows:
- Customers: customerNumber, customerName, contactLastName, contactFirstName, phone, addressLine1, addressLine2, city, state, postalCode, country, salesRepEmployeeNumber, creditLimit
- Employees: employeeNumber, lastName, firstName, extension, email, officeCode, reportsTo, jobTitle
- Offices: officeCode, city, phone, addressLine1, addressLine2, state, country, postalCode, territory
- Orderdetails: orderNumber, productCode, quantityOrdered, priceEach, orderLineNumber
- Orders: orderNumber, orderDate, requiredDate, shippedDate, status, comments, customerNumber
- Payments: customerNumber, checkNumber, paymentDate, amount
- Productlines: productLine, textDescription, htmlDescription, image
- Products: productCode, productName, productLine, productScale, productVendor, productDescription, quantityInStock, buyPrice, MSRP


In [25]:
res=agent_executer.run("what are the primary keys in each table?")

In [26]:
print(res)

- Customers table: customerNumber
- Employees table: employeeNumber
- Offices table: officeCode
- Orderdetails table: orderNumber, productCode
- Orders table: orderNumber
- Payments table: customerNumber, checkNumber
- Productlines table: productLine
- Products table: productCode


In [27]:
res=agent_executer.run("How each table is related to each other?")

In [28]:
print(res)

The tables are related as follows:
- Customers and Employees: The employees table has a foreign key reference to the salesRepEmployeeNumber in the customers table.
- Employees and Offices: The employees table has a foreign key reference to the officeCode in the offices table.
- Orderdetails and Orders: The orderdetails table has foreign key references to the orderNumber in the orders table.
- Orders and Customers: The orders table has a foreign key reference to the customerNumber in the customers table.
- Payments and Customers: The payments table has a foreign key reference to the customerNumber in the customers table.
- Products and Productlines: The products table has a foreign key reference to the productLine in the productlines table.


In [29]:
res=agent_executer.run("can you give some visualisation in pictorial form of how these tables are related to each other?")

In [31]:
print(res)

The tables in the database are related as follows: Customers -> Employees -> Offices, Orders -> Customers, OrderDetails -> Orders, OrderDetails -> Products, Payments -> Customers, Products -> ProductLines.


In [32]:
res=agent_executer.run("what are all the unique values in status column of orders table?")

In [33]:
print(res)

The unique values in the status column of the orders table are 'Shipped', 'Resolved', 'Cancelled', 'On Hold', 'Disputed', and 'In Process'.


In [34]:
agent_executer.run("give me a pie chart of how orders are distributed based on status ")

'The orders are distributed based on status as follows: Cancelled - 6, Disputed - 3, In Process - 6, On Hold - 4, Resolved - 4, Shipped - 303.'

In [49]:
agent_executer.run("how orders are distributed based on status")

"[('Cancelled', 6), ('Disputed', 3), ('In Process', 6), ('On Hold', 4), ('Resolved', 4), ('Shipped', 303)]"

In [50]:
res=agent_executer.run("What is the distinct count of status values from Orders table")

In [51]:
print(res)

The distinct count of status values from the Orders table is 6.
